# Tutorial 10 - Creating a model

In [Tutorial 9](./Tutorial%209%20-%20Changing%20the%20mesh.ipynb) we showed how to change the mesh using on of the built-in battery models in PyBaMM. In this tutorial we show how to create a simple model from scratch in PyBaMM.

As simple example, we consider the problem of linear diffusion on a unit sphere with a flux at the boundary that depends on the concentration. We solve
\begin{equation*}
  \frac{\partial c}{\partial t} = \nabla \cdot (\nabla c),
\end{equation*}
with the following boundary and initial conditions:
\begin{equation*}
  \left.\frac{\partial c}{\partial r}\right\vert_{r=0} = 0, \quad \left.\frac{\partial c}{\partial r}\right\vert_{r=1} = -j, \quad \left.c\right\vert_{t=0} = c_0,
\end{equation*}
where
\begin{equation*}
j = \left.j_0(1-c)^{1/2}c^{1/2}\right\vert_{r=1}.
\end{equation*}
Here $c_0$ and $j_0$ are parameters we can control. In this example we will assume that everything is non-dimensional and focus on how to set up and solve the model rather than any specific physical interpretation.

In [1]:
#%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm

## Setting up the model

First we load an empty model. We use the `BaseModel` class that sets up all the basic framework on which our model will be built.

In [2]:
model = pybamm.BaseModel()

We then define our variables and parameters using the `Variable` and `Parameter` classes, respectively. Since we are solving a PDE we need to tell PyBaMM the domain each variable belongs to so that it can be discretised in space in the correct way. This is done by passing the keyword argument `domain`, and in this example we choose the domain "particle".

In [3]:
c = pybamm.Variable("Concentration", domain="particle")
c0 = pybamm.Parameter("Initial concentration")
j0 = pybamm.Parameter("Flux parameter")

We then state out governing equations. In PyBaMM we distinguish between Ordinary Differential Equations of the form $dy/dt = \text{rhs}$ and Algebraic Equations of the form $f(y) = 0$. The model equations are stored in dictionaries where the key is the variable and the value is the rhs for ODEs and the residual ($f(y)$) for algebraic equations.

Sometime it is useful to define intermediate quantities in order to express the governing equations more easily. In this example we define the flux, then define the rhs to be minus the divergence of the flux. The equation is then added to the dictionary `model.rhs`

In [4]:
N = -pybamm.grad(c)  # define the flux
dcdt = -pybamm.div(N)  # define the rhs equation

model.rhs = {c: dcdt}  # add the equation to rhs dictionary with the variable as the key 

Next we add the necessary boundary and initial conditions to the model. These are also stored in dictionaries called `model.boundary_conditions` and `model.initial_conditions`, respectively. 

In [21]:
# boundary conditions 
c_surf = pybamm.surf(c)  # concentration at the surface of the sphere
j = j0 * (1 - c_surf) ** (1 / 2) * c_surf ** (1 / 2)  # prescribed boundary flux
model.boundary_conditions = {c: {"left": (0, "Neumann"), "right": (-j, "Neumann")}}

# initial conditions 
model.initial_conditions = {c: c0}

We can add any variables of interest to the dictionary `model.variables`. These can simply be the variables we solve for (in this case $c$) or any other user-defined quantities.

In [22]:
model.variables = {
    "Concentration": c,
    "Surface concentration": c_surf,
    "Flux": N,
    "Boundary flux": j,
}

## Setting up the geometry and mesh


In [23]:
r = pybamm.SpatialVariable("r", domain=["particle"], coord_sys="spherical polar")
geometry = {"particle": {r: {"min": 0, "max": 1}}}

submesh_types = {"particle": pybamm.Uniform1DSubMesh}
var_pts = {r: 20}
mesh = pybamm.Mesh(geometry, submesh_types, var_pts)

spatial_methods = {"particle": pybamm.FiniteVolume()}

## Solving the model

In [30]:
parameter_values = pybamm.ParameterValues({
    "Initial concentration": 0.9,
    "Flux parameter": 0.8,
})

solver = pybamm.ScipySolver()

sim = pybamm.Simulation(
        model,
        geometry=geometry,
        parameter_values=parameter_values,
        submesh_types=submesh_types,
        var_pts=var_pts,
        spatial_methods=spatial_methods,
        solver=solver,
)

In [31]:
sim.solve([0, 1])

In [32]:
sim.plot(model.variables.keys())

2022-07-08 15:05:41.704 - [WARNING] processed_variable.get_spatial_scale(521): No length scale set for particle. Using default of 1 [m].
2022-07-08 15:05:41.720 - [WARNING] processed_variable.get_spatial_scale(521): No length scale set for particle. Using default of 1 [m].


interactive(children=(FloatSlider(value=0.0, description='t', max=1.0, step=0.01), Output()), _dom_classes=('w…